In [2]:
import numpy as np 
import os
from classy import Class
import timeit
import multiprocessing as mp

In [3]:
M_sol = 1.989e30 # solar mass in kg
Mpc = 3.086e22 # Mpc in m
c=np.logspace(-11.3,-5, 40)
kvec = np.logspace(-4,np.log10(100),100)
measured_omegacdm=.1188


def trap(f,x):
    integral = 0
    for i in range(1,len(f)):
        integral += 0.5 * (f[i] + f[i-1]) * (x[i] - x[i-1])
    return integral
def Bella(k_vec,Pk_vec):
    
    def R(M):
        c = 2.5
        G = 6.67e-11 # m^3 / kg / s^2
        H100 = 100 * (1000/Mpc) # km/s/Mpc -> 1/s
        omegah2 = 0.1188

        rhobar = omegah2 * 3 * H100**2 / (8 * np.pi * G) / M_sol * Mpc**3 # kg / m^3 -> M_sol / Mpc^3
        return (3 * M / (4 * np.pi * rhobar * c**3))**(1/3) # Mpc
    
#    def W(k,R):
#        return 3 / (k * R)**3 * (np.sin(k*R) - 3 * np.cos(k*R))
    def W(k,R):
        if np.isscalar(k):
            if k * R > 1:
                return 0
            else:
                return 1
        else:
            result = np.zeros(len(k))
            for i in range(len(k)):
                if k[i] * R < 1 and k[i+1] * R > 1:
                    result[i] = (1 - k[i] * R)/(k[i+1]*R - k[i]*R)
                elif k[i] * R < 1:
                    result[i] = 1
            return result

    k_vals = np.array(k_vec)
    Pk_vals = np.array(Pk_vec)
    def S(M):
        Rv = R(M)
        
        integrand = k_vals**2 * Pk_vals * W(k_vals, Rv)**2 / (2 * ( np.pi**2))
        return trap(integrand,k_vals)
    
    
    
    M0 = 3.2e12 # M_sol / h
    P_spline  = sp.CubicSpline(k_vals,Pk_vals)
    def dNdlnM(M):
        Rv = R(M)
        return 1 / 44.5 / (6 * np.pi**2) * (M0 / M) / Rv**3 / np.sqrt(2 * np.pi * ( S(M) - S(M0))) * P_spline(1/Rv)
    
    lnM_vals = np.linspace(np.log(1e8),np.log(M0))
#    print("R",R(np.exp(lnM_vals)))
    sv = np.zeros(len(lnM_vals)-1)
    for i in range(len(sv)):
        sv[i] = S(np.exp(lnM_vals[i]))- S(np.exp(lnM_vals[-1]))
    
#    plt.figure()
#    plt.semilogy(lnM_vals[:-1],sv)
#    plt.show()
#    print(sv)
    integrand = np.zeros(len(lnM_vals)-1)
    for i in range(len(integrand)):
        integrand[i] = dNdlnM(np.exp(lnM_vals[i]))
#    print(integrand)
#    plt.figure()
#    plt.semilogy(lnM_vals[:-1],integrand)
#    plt.show()
    return trap(integrand,lnM_vals[:-1])

In [4]:
##test code using bellas old code from AAS Poster Notebook 
def make_Pk(fn,OmegaSh2):
    
    if OmegaSh2 > measured_omegacdm:
        print("Cannot use Omega_s h^2 > {}".format(measured_omegacdm))
        return

    commonsettings = {
                  'N_ncdm':1,
                  'use_ncdm_psd_files': 1,
                  'ncdm_psd_filenames': fn,
                  'm_ncdm': 7100,
                  'omega_cdm': measured_omegacdm - OmegaSh2,
                  'omega_ncdm': OmegaSh2,
                  'T_ncdm':0.7,
                  'output':'mPk',
                  'P_k_max_1/Mpc':100.0,
                  # The next line should be uncommented fgor higher precision (but significantly slower running)
                  'ncdm_fluid_approximation':3,
                  # You may uncomment this line to get more info on the ncdm sector from Class:
                  'background_verbose':1,
                  'Maximum q':100,
                  'A_s': 2.215e-9
                 }


    # array of k values in 1/Mpc


    NH = Class()
    NH.set(commonsettings)

    NH.compute()
    pkNH = []
    
    global sigma8 
    sigma8 = NH.sigma8()
    
    return sigma8

        #MPk- Matter Power Spectrum
    for k in kvec:
        pkNH.append(NH.pk(k,0.))
    h = NH.h()
    NH.struct_cleanup()

    #np.save(fn+'-Pk',np.array(pkNH))
    #np.save(fn+'-k',kvec/h)

In [5]:
path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + '1x0.0003x3e-12/Omegahh.npy'

path2  = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + '1x0.0003x3e-12/Spec'

In [6]:
file = np.load(path)
print(file)

0.0280563724602256


In [7]:
##sigma8 value outside of the bump 
make_Pk(path2,0.0280563724602256)

0.8419535465841379

In [8]:
pathbump = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + '1x0.0006x4e-12/Omegahh.npy'

path_to_bump = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + '1x0.0006x4e-12/Spec'

In [9]:
file2 = np.load(pathbump)
print(file2)

0.057079774860023844


In [10]:
##sigma8 value inside the bump 
make_Pk(path_to_bump,0.057079774860023844)

1.3052618005752221

In [11]:
##find the high sigma values

high_sigma8 = [] 

def highsigma():
        
    dirr = 'powerspec' + '/' + 'PaperRESULTS copy'
    
    for x in os.listdir(dirr):
        
        path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + x + '/Omegahh.npy'
        path2 = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' + x + '/Spec'
        
        if os.path.exists(path2):
            
            tmp_fn = np.load(path)
            try:
                
                make_Pk(path2,tmp_fn)
            
                if sigma8 > 0.848365:
                    high_sigma8.append(sigma8)
            except:
                print(dirr + '/' + x )
        
highsigma()        

Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/1x0.00075x4e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.00075x3e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.0008x5.5e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/1x0.0

Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.00065x9e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.0008x3e-12
powerspec/PaperRESULTS copy/0.1x0.00065x5e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
C

Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.0007x3.5e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0

Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
powerspec/PaperRESULTS copy/10x0.0007x4e-12
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1188
Cannot use Omega_s h^2 > 0.1

In [12]:
print(high_sigma8)

[0.8495452921426766, 1.5633222157243183, 0.849939969822348, 0.8510509837649959, 0.8523077043811074, 0.8523077043811074, 0.8523077043811074, 0.8523077043811074, 0.8523077043811074, 0.8523077043811074, 0.8505240912061673, 0.8505240912061673, 0.8505240912061673, 0.8490343872760041, 0.8490343872760041, 0.8490343872760041, 3.9301742019889074, 0.8487499184488639, 0.8495584201625321, 0.8495584201625321, 0.848740260670893, 0.848740260670893, 0.848740260670893, 0.848740260670893, 0.848733492544677, 1.032133303657119, 1.032133303657119, 1.032133303657119, 1.078144721911126, 1.078144721911126, 1.078144721911126, 1.078144721911126, 0.851906107689026, 0.852333897641323, 0.852057108376316, 1.0285364446275875, 0.8489228359135073, 0.8489228359135073, 0.8489228359135073, 0.8489228359135073, 0.8489228359135073, 0.8497893147027163, 0.8497893147027163, 0.8485945742488746, 0.8489136266825899, 0.8489136266825899, 0.8491486931728652, 0.8516512321624393, 0.8516608440639871, 0.8516608440639871, 0.8516608440639

In [13]:
len(high_sigma8)

445

In [14]:
import statistics

sorted_sigma = sorted(high_sigma8)

copy_sigma = high_sigma8.copy()

new_sigma = sorted_sigma[295:]

value = []
for x in new_sigma:
    val = x - 0.848365
    value.append(val)
value

list_average = statistics.mean(value)
print(list_average)

0.9613422083275792


In [21]:
sorted_sigma

[0.8483755032220042,
 0.8483755032220042,
 0.848412192507071,
 0.848412192507071,
 0.8484865494098941,
 0.848495362299857,
 0.8485043120754512,
 0.8485043120754512,
 0.8485043120754512,
 0.848522293147364,
 0.8485945742488746,
 0.848620256864178,
 0.8486243647544899,
 0.8486684079346762,
 0.8487259559601135,
 0.8487259559601135,
 0.8487259559601135,
 0.8487259559601135,
 0.848733492544677,
 0.848740260670893,
 0.848740260670893,
 0.848740260670893,
 0.848740260670893,
 0.8487499184488639,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487541034699496,
 0.8487557361118301,
 0.8487557361118301,
 0.8487557361118301,
 0.8487557361118301,
 0.8487557361118301,
 0.8488217286785319,
 0.8488350791590313,
 0.8489136266825899,
 0.8489136266825899,
 0.8489228359135073,
 0.8489228359135073,
 0.8489228359135073,
 0.8489228359135073,
 0.8489228359135073,
 0.8489240403114073,
 0.8489323825462493,
 0

In [16]:
# path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' #this is the file path, should call path

# def sigma_test(file_name, omega, x):

#     #this line is a path to the '''Spec- ''' files 
#     path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/'

#     #initializing the '''Spec-''' files one at a time through loop
#     #also loading the """Omegahh.npy"""" from this folder so it can be placed into common settings 

#     #for folder in os.listdir(file):
#         #fn       = file + folder + '/' + 'Spec' 
#         #OmegaSh2 = np.load(file + folder + '/' + 'Omegahh.npy')

#     #common settings are some of the settings that we want to tell the computer to keep constant and not default
#     #for this particualr calculation

#     commonsettings = {
#               'N_ncdm':1,
#               'use_ncdm_psd_files': 1,
#               'ncdm_psd_filenames': file_name,
#               'm_ncdm': 7100,
#               'omega_cdm': measured_omegacdm - omega,
#               'omega_ncdm': omega,
#               'T_ncdm':0.7,
#               'output':'mPk',
#               'P_k_max_1/Mpc':100.0,
#               # The next line should be uncommented fgor higher precision (but significantly slower running)
#               'ncdm_fluid_approximation':3,
#               # You may uncomment this line to get more info on the ncdm sector from Class:
#               'background_verbose':1,
#               'Maximum q':100
#              }


#     # array of k values in 1/Mpc

#     #create an instance of this Class named "NH" or "Nuetrino Mass Heirarchy"
#     NH = Class()
#     #use method .set() to 
#     NH.set(commonsettings)

#     #use method .compute() to get data for my specific 'Spec-' file

#     NH.compute()

#     #storing all of the 'numbers' that we can get from the .compute() calculation, into a dictionary that I will 
#     #save into a .npz file and later use to build plots so we can posisbly find the "bump" in the data

#     values_dict = {
#                     'Sigma8': NH.sigma8() - 0.9613422083275792,
#                     'Sigma8_cb': NH.sigma8_cb() -0.9613422083275792,
#                     }
#     #this method clears the commonsettings out of the set method, so that we can load a new file 
#     NH.struct_cleanup() 
    
#     #saving the dictionary that we created above to a .npz file
#     np.savez(file = x + '.npz', sigma_values = values_dict)

In [17]:
list_sigma = []
ps = mp.Pool(4)
res = ps.starmap(sigma_test,list_sigma)

TypeError: starmap() missing 1 required positional argument: 'iterable'

Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nu_tau/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nu_tau/anac

In [ ]:
## this is the function that I want to send thorugh the for looop after I figure out how to get the sim to run, 
## and use the mul;tiprocessing method in python 
# ## I have tested a few files, and the wall time for using nh.compuyte is ~ .40s 


# ##possible loop that I am going to use for the larger selection of files
# for folder in os.listdir(file):
#     fn       = file + folder + '/' + 'Spec' 
#     OmegaSh2 = np.load(file + folder + '/' + 'Omegahh.npy')

# def load(fn,OmegaSh2):
#     #this line is a path to the '''Spec- ''' files 
#     file = 'powerspec' + '/' + 'PaperRESULTS copy'


        
#     #common settings are some of the settings that we want to tell the computer to keep constant and not default
#     #for this particualr calculation
    
#     commonsettings = {
#               'N_ncdm':1,
#               'use_ncdm_psd_files': 1,
#               'ncdm_psd_filenames': fn,
#               'm_ncdm': 7100,
#               'omega_cdm': measured_omegacdm - OmegaSh2,
#               'omega_ncdm': OmegaSh2,
#               'T_ncdm':0.7,
#               'output':'mPk',
#               'P_k_max_1/Mpc':100.0,
#               # The next line should be uncommented fgor higher precision (but significantly slower running)
#               'ncdm_fluid_approximation':3,
#               # You may uncomment this line to get more info on the ncdm sector from Class:
#               'background_verbose':1,
#               'Maximum q':100,
#               'sigma8': 0.848365
#              }


#     # array of k values in 1/Mpc

#     #create an instance of this Class named "NH" or "Nuetrino Mass Heirarchy"
#     NH = Class()
#     #use method .set() to 
#     NH.set(commonsettings)
    
#     #use method .compute() to get data for my specific 'Spec-' file
    
#     NH.compute()
    
#     #storing all of the 'numbers' that we can get from the .compute() calculation, into a dictionary that I will 
#     #save into a .npz file and later use to build plots so we can posisbly find the "bump" in the data
    
#     values_dict = {
#                     'age': NH.age(),
#                     'h': NH.h(),
#                     'n_s': NH.n_s(),
#                     'Neff': NH.Neff(),
#                     'Omega0_cdm': NH.Omega0_cdm(),
#                     'Omega0_k': NH.Omega0_k(),
#                     'Omega0_m': NH.Omega0_m(),
#                     'Omega_b': NH.Omega_b(),
#                     'omega_b': NH.omega_b(),
#                     'Omega_g': NH.Omega_g(),
#                     'Omega_lambda': NH.Omega_Lambda(),
#                     'Omega_m':NH.Omega_m(),
#                     'Omega_r': NH.Omega_r(),
#                     'rs_drag': NH.rs_drag(),
#                     'Sigma8': NH.sigma8(),
#                     'Sigma8_cb': NH.sigma8_cb(),
#                     'T_cmb': NH.T_cmb(),
#                     'tau_reio': NH.tau_reio(),
#                     'theta_s_100': NH.theta_s_100(),
#                     'theta_star_100': NH.theta_star_100(),
            
#                     }
#     #this method clears the commonsettings out of the set method, so that we can load a new file 
#     NH.struct_cleanup() 
    
#     #saving the dictionary that we created above to a .npz file
#     np.savez()

In [ ]:

# path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/' #this is the file path, should call path

# def test(file_name, omega):

#     #this line is a path to the '''Spec- ''' files 
#     path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/'

#     #initializing the '''Spec-''' files one at a time through loop
#     #also loading the """Omegahh.npy"""" from this folder so it can be placed into common settings 

#     #for folder in os.listdir(file):
#         #fn       = file + folder + '/' + 'Spec' 
#         #OmegaSh2 = np.load(file + folder + '/' + 'Omegahh.npy')

#     #common settings are some of the settings that we want to tell the computer to keep constant and not default
#     #for this particualr calculation

#     commonsettings = {
#               'N_ncdm':1,
#               'use_ncdm_psd_files': 1,
#               'ncdm_psd_filenames': file_name,
#               'm_ncdm': 7100,
#               'omega_cdm': measured_omegacdm - omega,
#               'omega_ncdm': omega,
#               'T_ncdm':0.7,
#               'output':'mPk',
#               'P_k_max_1/Mpc':100.0,
#               # The next line should be uncommented fgor higher precision (but significantly slower running)
#               'ncdm_fluid_approximation':3,
#               # You may uncomment this line to get more info on the ncdm sector from Class:
#               'background_verbose':1,
#               'Maximum q':100
#              }


#     # array of k values in 1/Mpc

#     #create an instance of this Class named "NH" or "Nuetrino Mass Heirarchy"
#     NH = Class()
#     #use method .set() to 
#     NH.set(commonsettings)

#     #use method .compute() to get data for my specific 'Spec-' file

#     NH.compute()

#     #storing all of the 'numbers' that we can get from the .compute() calculation, into a dictionary that I will 
#     #save into a .npz file and later use to build plots so we can posisbly find the "bump" in the data

#     values_dict = {
#                     'age': NH.age(),
#                     'h': NH.h(),
#                     'n_s': NH.n_s(),
#                     'Neff': NH.Neff(),
#                     'Omega0_cdm': NH.Omega0_cdm(),
#                     'Omega0_k': NH.Omega0_k(),
#                     'Omega0_m': NH.Omega0_m(),
#                     'Omega_b': NH.Omega_b(),
#                     'omega_b': NH.omega_b(),
#                     'Omega_g': NH.Omega_g(),
#                     'Omega_lambda': NH.Omega_Lambda(),
#                     'Omega_m': NH.Omega_m(),
#                     'Omega_r': NH.Omega_r(),
#                     'rs_drag': NH.rs_drag(),
#                     'Sigma8': NH.sigma8(),
#                     'Sigma8_cb': NH.sigma8_cb(),
#                     'T_cmb': NH.T_cmb(),
#                     'tau_reio': NH.tau_reio(),
#                     'theta_s_100': NH.theta_s_100(),
#                     'theta_star_100': NH.theta_star_100(),

#                     }
#     #this method clears the commonsettings out of the set method, so that we can load a new file 
#     NH.struct_cleanup() 

#     #saving the dictionary that we created above to a .npz file
#     np.savez(file = 'TEST' + '.npz', Class_values = values_dict)

In [ ]:
# test_omega = np.load(path + '0.1x0.0001x1.25e-11' + '/' + 'Omegahh.npy')
# test_file  = path +  '0.1x0.0001x1.25e-11' + '/' + 'Spec' 

# test(test_file, test_omega).

In [ ]:
# ##test cell for the defalut parameters of Class
# ##
# tested_file = np.load('TEST.npz', allow_pickle = True)
# tested_file.files
# tested_file['Class_values'].item()
